# Requests + JSON using guide

Для работы с ***API***  чаще всего придётся пользоваться именно такой связок двух библиотек. 

Пожалуй начнём с их импорта

In [8]:
import json
import requests

Практически всегда (не помню когда не так) нам пригодится посылать `.get` запросы на сервер для получения ответа. Давайте разберём работу, например, запрос к сайту с [картами](https://en.mapy.cz/)

Один из внутренних запросов ( https://pro.mapy.cz/suggest/?count=5&phrase=metro&lon=37.2652&lat=55.6784&zoom=7&enableCategories=1&lang=en&personalize=1&includeNonEntityTypes=1&userLat=55.6784&userLon=37.2652&userAccuracy=20000 ) содержит внутренние параметры:

```python
- count=5
- phrase=metro
- lon=37.2652
- lat=55.6784
- zoom=7
- enableCategories=1
- lang=en
- personalize=1
- includeNonEntityTypes=1
- userLat=55.6784
- userLon=37.2652
- userAccuracy=20000
```

*Интереса и опыта ради можете сами найти эту ссылку в панели* `Network`*, и покрутить значения с целью понять, за что отвечает каждый из параметров (хотя тут названия говорят сами за себя)*

In [2]:
url = 'https://pro.mapy.cz/suggest/?count=5&phrase=metro&lon=37.2652&lat=55.6784&zoom=7&enableCategories=1&lang=en&personalize=1&includeNonEntityTypes=1&userLat=55.6784&userLon=37.2652&userAccuracy=20000'
page = requests.get(url)

print(page)

<Response [200]>


`<Response [200]>` означает, что запрос успешно обработан и получен ответ. За дополнительной информацией обращайтесь [сюда](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP)

Взглянем-ка на то, что вернул сервер



In [7]:
content = page.text
print(type(content), content[:100] + '...', sep ='\n\n')

<class 'str'>

{"deletedFromBack":0,"hasGeo":0,"hasService":0,"id":"8f2ea956-1f15-4556-8319-b36c3fac1f14","result":...


Это очень похоже на словарь, однако сейчас это строка. Чтобы это дело исправить нужно лишь понять, что перед нами JSON-файл в естественной среде обитания, а его хорошо парсит и превращает в классический словарь/список метод библиотеки json.

Воспользуемся методом <mark>.loads</mark> (load-string) для того чтобы работать с возвращаемым значением

In [20]:
data = json.loads(content)

print(type(data), data.keys(), sep='\n\n')

<class 'dict'>

dict_keys(['deletedFromBack', 'hasGeo', 'hasService', 'id', 'result', 'sortToUserOnCategory'])


Теперь с этими данными можно работать как с обычным словарём.

In [21]:
res = data['result'][0]

print(res)

res['userData']['bbox'] = (None,None)

print(res)

{'category': 'pubtran_osm', 'highlight': [], 'sentence': '', 'userData': {'bbox': [55.63261902819228, 37.32790265526835, 55.636422066867894, 37.339131596319845], 'correctedResult': False, 'country': '', 'district': '', 'evidenceNumber': '', 'hasAddress': False, 'highlight': [0, 5], 'highlightSecond': [], 'houseNumber': '', 'iconType': 'default', 'id': 1071044353, 'img': '', 'importance': 0.140161, 'latitude': 55.634520593672576, 'longitude': 37.33351712579409, 'mmid': '', 'mmsource': '', 'mmtype': '', 'muniId': '', 'municipality': '', 'nuts': '', 'poiType': 'poi6001', 'poiTypeId': 6001, 'popularity': 0, 'premiseIds': [], 'quarter': '', 'region': '', 'relevance': 0.7496, 'ruianId': 0, 'source': 'osm', 'street': '', 'streetNumber': '', 'suggestFirstRow': 'Metro Rasskazovka', 'suggestSecondRow': 'Bus stop, Vnukovskoye Settlement, Russia', 'suggestThirdRow': '', 'ward': '', 'wikiId': 'Q0', 'zipCode': ''}}
{'category': 'pubtran_osm', 'highlight': [], 'sentence': '', 'userData': {'bbox': (No

Ну и сохраняется всё это дело обычным методом <mark>json.dump</mark>. Например 

In [22]:
with open('backup.json', 'w') as gate:
    # json.dump(What to save?, Where to save?)
    json.dump(res, gate)

Типы данных, что могут храниться в JSON:
- Список
- Словарь
- Число
- True/False
- Строка
- null

Всё что к этому не относится (как кортеж None'ов в последнем примере), будет сконвертированно в один из возможных типов (в список null'ов), если это возможно

---
### P.s : работа над ошибками
Внешнее сходство дефолтного отображения списка и *JSON*-файла не позволяет вам  влоб сохранять файлы без `json.dump`.

А вот и демонстрация того - почему:

In [11]:
# Сперва определим наши данные
my_data = {
    'bool': True,
    'string': 'hell yeah!',
    'int' : 7,
    'float': 3.1415,
    'list': [False, 'a', 2, 3.0],
    'dict': {'key': 'value'},
    'None': None
}

In [12]:
# Обычная запись словаря в файл

with open('get_error.json', 'w') as gate:
    gate.write(str(my_data))

with open('get_error.json', 'r') as source:
    print(source.read())

{'bool': True, 'string': 'hell yeah!', 'int': 7, 'float': 3.1415, 'list': [False, 'a', 2, 3.0], 'dict': {'key': 'value'}, 'None': None}


In [13]:
with open('skip_error.json', 'w') as gate:
    json.dump(my_data,gate)
    
with open('skip_error.json', 'r') as source:
    print(source.read())
    

{"bool": true, "string": "hell yeah!", "int": 7, "float": 3.1415, "list": [false, "a", 2, 3.0], "dict": {"key": "value"}, "None": null}


Казалось-бы... что такого? Кажется ответы совпадают =/

А теперь давайте попробуем считать первый файл методом `json.loads`

In [10]:
with open('get_error.json', 'r') as source:
    data = json.loads(source.read())

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

И виной тому следующие пункты:

**"Двойные!"** : 
- В *JSON* файлах для разграничения строк используются только двойные кавычки

**Регистр ИмЕеТ значениЕ*** :
- Вместо привычных для нас True/False в файл записываются true/false.

**None != null** : 
- для неопределённых значений в *JSON* используется понятие null, а не None.

---

Теперь зная это мы можем, конечно, научиться записывать в файл без использования библиотеки:

In [14]:
def py_to_json(data):
    content = str(data)
    content = content.replace('True','true')
    content = content.replace('False','false')
    content = content.replace('None','null')
    content = content.replace('\'','"')
    return content

Но и с такой функцие проблемы рано или поздно появятся

In [17]:
data = {
    "who's your daddy?": "Nonessential function",
}

# Ну вы поняли =)

with open('get_error.json', 'w') as gate:
    gate.write(py_to_json(data))

with open('get_error.json', 'r') as source:
    data = json.loads(source.read())

JSONDecodeError: Expecting ':' delimiter: line 1 column 7 (char 6)